In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import max_functions as mf

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

path = r'D:\UniFRK\SEXTANT_sept2018\S-TFMoX\enant=S_hel=+1_KE=11.5eV' # use your path
all_files = glob.glob(path + "/*.dat")

li = []
cosphi_th = []
colnames=["phi","cos(theta)","value"] 

# how to load multiple files http://jonathansoma.com/lede/foundations-2017/classes/working-with-many-files/class/
for filename in all_files:
    df = pd.read_csv(filename, delimiter=r"\s+", names=colnames, header=None) 
    # or delim_whitespace=True, it is faste
    # r"\s+" is a regex (regular expression)
    df["filename"]=filename.split("\\")[-1].split(".")[0] # adding a column with the file name
    df["filename"]=df["filename"].str.replace(" ","") # corrects for spaces
    cosphi_th.append((int(df["filename"].str.split("_")[0][0]),int(df["filename"].str.split("_")[0][1])))
    df["filename"]=df["filename"].astype("category")
    li.append(df) # a unique DataFrame

cosphi_th=np.array(cosphi_th)
frame = pd.concat(li, axis=0)
frame_srt=frame.groupby("filename")

In [ ]:
frame.loc[frame["filename"] == "1_1"] # first way of selecting the right file
frame_set = frame.set_index("filename") # build a multiindex using the categories of filename

In [ ]:
# Single 2D maps from pandas without tranforming into numpy
frame.loc[frame["filename"] == "1_1"].plot.hexbin(x="phi", y="cos(theta)", C="value", sharex=False, gridsize=50)

In [ ]:
# converting to numpy arrays to plot
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)
counts = frame.loc[frame["filename"] == "1_1"].iloc[:,2].to_numpy() # values

print(phi.shape, ctheta.shape, counts.shape)
print(type(phi), type(ctheta), type(counts))
counts_matrix = counts.reshape(200,100) # reshape(rows=x,columns=y)

print("number of ROWS (x) = ", len(counts_matrix[:,1]), "number of COLUMNS (y for reshaping) = ", len(counts_matrix[1,:]))

In [ ]:
# plt.imshow(z_matrix)
plt.imshow(counts_matrix.T,extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], aspect=180)
ax = plt.axes()
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
plt.colorbar()

In [ ]:
# countour plot
cmap = mpl.cm.viridis
cmap.set_under('w')


fig1, ax = plt.subplots(constrained_layout=True)

CS = ax.contourf(phi,ctheta,counts_matrix.T, levels=20, extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], vmin=0.0001)
ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
fig1.colorbar(CS)

# OLD WAY OF PROCEDING!! Note unique values and extent, all the 72 plots
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)

fig = plt.figure(figsize=(50, 24),dpi=80) # figsize in inche by defauls: 1 in = 2.8 cm
for filename,group,i in zip(frame_srt,frame_srt,range(1,73)) :
    counts_str = group[1]["value"].to_numpy()
    counts_matrix = counts_str.reshape(200,100)
    ax = fig.add_subplot(6, 12, i)
    ax.contourf(phi,ctheta,counts_matrix, levels=20, extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], vmin=0.0001)
    ax.text(i, 1+i//12, i, ha="center", va="center", color="red")

fig.tight_layout()
# plt.show()

In [ ]:
# all the 72 plots
# the theoretical MFPAD has the form 1_1 ... 1_12, 2_1 ... 2_12 therefore it is ok the (6,12) subplot
# 6 levels of cos(theta) (y), 12 levels of phi (x)
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)

# fig = plt.figure(figsize=(50, 24),dpi=80) # figsize in inches by defauls: 1 in = 2.8 cm
fig = plt.figure(figsize=(25, 12),dpi=40) # downsized
for filename,group,i in zip(frame_srt,frame_srt,range(72)):
    counts_str = group[1]["value"].to_numpy()
    ax = fig.add_subplot(6, 12, i+1)
    ax.contourf(phi.reshape(200,100),ctheta.reshape(200,100),counts_str.reshape(200,100), levels=20, vmin=0.0001)
    ax.text(i+1, i//12+1, cosphi_th[i], ha="center", va="center", color="red")

fig.tight_layout()
# plt.show()

# alterative methods for subplots: not working well, lets marks

fig, axs = plt.subplots(2,5, figsize=(50, 24), sharex=True, sharey=True)
axs = axs.ravel()

# all the 72 plots
phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].unique() # phi
ctetha = frame.loc[frame["filename"] == "1_1"].iloc[:,1].unique() # cos(theta)

for filename,group,i in zip(frame_srt,frame_srt,range(1,73)) :
    counts_str = group[1]["value"].to_numpy()
    counts_matrix = counts_str.reshape(200,100)
    ax = fig.add_subplot(6, 12, i)
    ax.contourf(phi,ctheta,counts_matrix.T, levels=20, extent=[phi[0],phi[-1],ctheta[0],ctheta[-1]], vmin=0.0001)

fig.tight_layout()
plt.show()

In [ ]:
# all the 72 plots: no borders & sharing axis


phi = frame.loc[frame["filename"] == "1_1"].iloc[:,0].to_numpy() # phi
ctheta = frame.loc[frame["filename"] == "1_1"].iloc[:,1].to_numpy() # cos(theta)

# fig, axes = plt.subplots(6,12, figsize=(50, 24), sharex='col', sharey='row') figsize in inche by defauls: 1 in = 2.8 cm
fig, axes = plt.subplots(6,12, figsize=(25, 12), dpi=40, sharex='col', sharey='row') #downsized
fig.subplots_adjust(hspace = .5, wspace=.5)

for filename,group,ax in zip(frame_srt,frame_srt,axes.flatten()) :
    counts_str = group[1]["value"].to_numpy()
    counts_matrix = counts_str.reshape(200,100)
    ax.contourf(phi.reshape(200,100),ctheta.reshape(200,100),counts_str.reshape(200,100), levels=20, vmin=0.0001)
    
fig.tight_layout()
plt.show()